In [1]:
# Import modules
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers
import numpy as np

## Load Data

In [2]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [3]:
print("Shape of X_train: {}".format(X_train.shape))
print("Shape of y_train: {}".format(y_train.shape))
print("Shape of X_test: {}".format(X_test.shape))
print("Shape of y_test: {}".format(y_test.shape))

Shape of X_train: (50000, 32, 32, 3)
Shape of y_train: (50000, 10)
Shape of X_test: (10000, 32, 32, 3)
Shape of y_test: (10000, 10)


## Declare Model

In [4]:
i = keras.Input(shape=(32, 32, 3,))
x = layers.Conv2D(16, 3,padding='same',activation="relu")(i)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(32, 3,padding='same',activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(10,activation='softmax')(x)

model = keras.Model(inputs=i, outputs=x, name="mnist_model")
model.summary()

Model: "mnist_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                

## Custom Train Loop

In [5]:
@tf.function
def train_on_batch(X, y):
    with tf.GradientTape() as tape:
        yh = model(X, training=True)
        loss_value = loss(y, yh)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

@tf.function
def validate_on_batch(X, y):
    yh = model(X, training=False)
    loss_value = loss(y, yh)
    return loss_value

In [6]:
loss = tf.keras.losses.categorical_crossentropy
optimizer = tf.keras.optimizers.Adam(0.001)
batch_size = 1024
epochs = 10

train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=len(X_train)).batch(batch_size)
test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test)).shuffle(buffer_size=len(X_test)).batch(batch_size)

In [8]:
best_loss = 99999
for epoch in range(0, epochs):
    for batch, (X, y) in enumerate(train_data):
        train_on_batch(X, y)
        print('\rEpoch [%d/%d] Batch: %d%s' % (epoch + 1, epochs, batch, '.' * (batch % 10)), end='')

    val_loss = np.mean([np.mean(validate_on_batch(X, y)) for (X, y) in test_data])
    print('. Validation Loss: ' + str(val_loss))
    if val_loss < best_loss:
        model.save_weights('model.h5')
        best_loss = val_loss

Epoch [1/10] Batch: 48.......... Validation Loss: 1.5902557
Epoch [2/10] Batch: 48.......... Validation Loss: 1.4908419
Epoch [3/10] Batch: 48.......... Validation Loss: 1.429952
Epoch [4/10] Batch: 48.......... Validation Loss: 1.3814714
Epoch [5/10] Batch: 48.......... Validation Loss: 1.3311608
Epoch [6/10] Batch: 48.......... Validation Loss: 1.299958
Epoch [7/10] Batch: 48.......... Validation Loss: 1.268738
Epoch [8/10] Batch: 48.......... Validation Loss: 1.2375911
Epoch [9/10] Batch: 48.......... Validation Loss: 1.230054
Epoch [10/10] Batch: 48.......... Validation Loss: 1.2007405
